In [1]:
%%time
%reload_ext autoreload
%autoreload
%autoreload 2
%config Completer.use_jedi = False

import os
import json
import warnings
import pprint
import sys
import numpy as np
import pandas as pd
import torch

MAIN_PATH = os.getcwd().split("notebooks")[0]
sys.path.insert(0, MAIN_PATH)

CPU times: user 2.26 s, sys: 532 ms, total: 2.79 s
Wall time: 2.45 s


In [2]:
# Run a Federated Learning experiment
from data_loader.mnist import MnistDatasetManager
from models.mnist_model import MnistFullConnectModel
from server.base_server import BaseServer
from client.base_client import BaseClient
from experiments.base_experiment import BaseExperiment
from gradients.noise import GaussianNoiseGenerator
from metrics.classification import multiclass_accuracy

In [3]:
DATA_MANAGER = MnistDatasetManager(n_parties=2)


In [4]:
class DemoMnistExperiment(BaseExperiment):
    def __init__(self, client_num: int =2):
        self._init_server_clients(client_num)
        self._init_data(client_num)
    
    def _init_server_clients(self, client_num):
        model = MnistFullConnectModel
        self.clients = [BaseClient(model(), 
                                   client_id=idx, 
                                   noise_generator=GaussianNoiseGenerator())
                        for idx in range(client_num)]
        self.server = BaseServer(model())

    def _init_data(self, client_num):
        data_manager = MnistDatasetManager(n_parties=client_num)
        self.client_train_datas = data_manager.train_loaders
        self.valid_datas = data_manager.validation_loader
        self.test_data = data_manager.test_loader

    def evaluate_model(self, data):
        total_correct = 0
        total_sample_num = 0
        with torch.no_grad():
            for _, (inputs, target) in enumerate(data):
                predict_labels = self.server.predict(inputs)
                correct, sample_num = multiclass_accuracy(predict_labels, 
                                                          target)
                total_correct += correct
                total_sample_num += sample_num
                
        return total_correct / total_sample_num
        
    def get_validation_result(self):
        return self.evaluate_model(self.valid_datas)
    
    def get_test_result(self):
        return self.evaluate_model(self.test_data)
    
    def aggeragate(self):
        for client in self.clients:
            client.set_training_mode(for_gradient=False)
        self.server.aggeragate_model(self.clients)

In [5]:
EXPERIMENT = DemoMnistExperiment(client_num=2)

In [7]:
EXPERIMENT.run(epochs=10)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)